In [32]:
from models.MLP_dir import MLP_dir
from models.MLP_incr import MLP_incr
from data.materials import load_responses
from sklearn.model_selection import train_test_split
import torch
import numpy as np

def data_to_tensor(x:np.array):
    return torch.tensor(
        x.astype(np.float32),
        dtype=torch.float32
    ).unsqueeze(-1)

In [33]:
def train_model(model: MLP_dir | MLP_incr, mat_name, inp_name): 
    eps_list, sig_list = load_responses(mat_name,'random',inp_name,data_path='data')

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    y_train, y_tmp, u_train, u_tmp = train_test_split(y_list, u_list, test_size=0.3, random_state=42)
    y_val, y_test, u_val, u_test = train_test_split(y_tmp, u_tmp, test_size=0.5, random_state=42)

    model.fit(
        epochs=500,
        y_train=y_train, u_train=u_train,
        y_val=y_val,u_val=u_val,
        early_stopping=True,
        patience=20,
        min_delta=1e-6,
        verbose=True,
        shuffle=True,
    )

    return model.evaluate(y_test,u_test).item()

In [34]:
model = MLP_dir(2,8,2,weight_decay=1e-4)

test_err = train_model(
    model,
    mat_name = 'isotropic-swift',
    inp_name = 'pd_ms_42_200',
)

print(f"{test_err*100} %")

Epoch 1/500, Loss: 0.11229584366083145, Val Loss: 0.09320937680701415, LR: 0.001
Epoch 2/500, Loss: 0.0692615658044815, Val Loss: 0.0415644612784187, LR: 0.001
Epoch 3/500, Loss: 0.04784565046429634, Val Loss: 0.024403351762642463, LR: 0.001
Epoch 4/500, Loss: 0.03248769789934158, Val Loss: 0.016929359221830965, LR: 0.001
Epoch 5/500, Loss: 0.02184094861149788, Val Loss: 0.012248098741595943, LR: 0.001
Epoch 6/500, Loss: 0.015016420744359493, Val Loss: 0.00907316073620071, LR: 0.001
Epoch 7/500, Loss: 0.01114810910075903, Val Loss: 0.006989784336959322, LR: 0.001
Epoch 8/500, Loss: 0.009016027674078941, Val Loss: 0.0056344324567665655, LR: 0.001
Epoch 9/500, Loss: 0.007775492034852505, Val Loss: 0.0047086613757225376, LR: 0.001
Epoch 10/500, Loss: 0.0069667985662817955, Val Loss: 0.004037710345194985, LR: 0.001
Epoch 11/500, Loss: 0.006343997549265623, Val Loss: 0.003524378341777871, LR: 0.001
Epoch 12/500, Loss: 0.005827395711094141, Val Loss: 0.0031156231610414884, LR: 0.001
Epoch 13

In [35]:
def eval_model(model: MLP_dir | MLP_incr, mat_name, inp_type, inp_name):
    
    eps_list, sig_list = load_responses(
        mat_name,inp_type,inp_name,
        data_path='data'
    )

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    return model.evaluate(y_list,u_list).item()

In [36]:
mat_name = 'isotropic-swift'

eval_list = []
for inp_name in ['amplitude', 'cyclic', 'impulse', 'piecewise']:
    eval_list += [eval_model(model,mat_name,'static',inp_name)]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

for inp_name in ['bl_ms_42_200', 'gp_42_200', 'pd_ms_42_200', 'rw_42_200']:
    eval_list += [eval_model(model,mat_name,'random',inp_name)]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

print(f'\nMean: {np.mean(eval_list)*100:02f} %')

amplitude: 6.997047 %
cyclic: 9.907893 %
impulse: 3.804880 %
piecewise: 22.168332 %
bl_ms_42_200: 1.178019 %
gp_42_200: 2.452763 %
pd_ms_42_200: 0.906303 %
rw_42_200: 4.708946 %

Mean: 6.515523 %


(0, 200, False): 3.656732 %

(1e-4, 200, False): 3.622211 %

(1e-4, 200, True): 3.656732 %

(1e-2, 200, False): 4.618025 %

(1e-4, 1000, False): 64.602013 %

(1e-2, 1000, False): 6.192588 %